In [2]:
!pip install transformers
!pip install datasets
!pip install ipywebrtc
!pip install soundfile
!pip install pydub
!pip install ffmpeg-python
!pip install git+https://github.com/speechbrain/speechbrain.git@develop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.4 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 49.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

# Automatic Speech Recognition

In [1]:
import torch

def get_device():
  device='cpu'
  if torch.cuda.is_available():
    device='cuda:0'
  elif torch.backends.mps.is_available():
    device='mps'
  else:
    device='cpu'
  return 'cpu'

device = get_device()
print (device)

cpu


In [2]:
import os
import torch
from speechbrain.inference.classifiers import EncoderClassifier
import torchaudio
import torchaudio.transforms as T
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan


In [3]:
model_name = "speechbrain/spkrec-xvect-voxceleb"

speaker_classifier = EncoderClassifier.from_hparams(
    source=model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", model_name)
)

def compute_speaker_embedding(audio_data):
    with torch.no_grad():
        embeddings = speaker_classifier.encode_batch(torch.tensor(audio_data))
        embeddings = torch.nn.functional.normalize(embeddings, dim=2)
        embeddings = embeddings.squeeze().cpu().numpy()
    return embeddings

In [4]:
waveform, sample_rate = torchaudio.load('/Users/premtimsina/Downloads/audio_sample2.wav')

# If audio is stereo, convert to mono
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0)

# Resample the waveform to 16kHz
resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)

# Squeeze the tensor to remove the channel dimension
waveform = waveform.squeeze()


In [5]:
speaker_emb=compute_speaker_embedding(waveform)
speaker_emb=torch.tensor(speaker_emb).reshape(-1,512)
print(speaker_emb.shape)

torch.Size([1, 512])


/var/folders/dt/v286l_1506x752dg9dwh_yl40000gn/T/ipykernel_20384/3839088228.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings = speaker_classifier.encode_batch(torch.tensor(audio_data))


In [6]:

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [7]:
inputs = processor(text="This is Harry. I live in New York City", return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_emb, vocoder=vocoder)

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [8]:
from IPython.display import Audio

Audio(speech, rate=16000)